# Actor-critic methods

## Principe d'actor-critic

L'idée est d'avoir deux fonctions:

- L'une, que l'on va appeler l'actor, qui va décider des actions à prendre: il s'agit de la fameuse fonction ${\pi(s)}$ du RL, qui à chaque état ${s}$ associe une distribution de probabilité sur les actions, une action étant plus probable pour cet état si il est plus pertinent de l'effectuer. 
- L'autre, la critique, va dire à l'actor si ça décision a été bonne ou non et comment l'améliorer. En clair, le critic permet de mettre à jour l'actor. Il s'agit par exemple des fonctions ${V(s)}$ (value function) qui à chaque état associe une valeur d'espérance de récompense, ou ${Q(s, a)}$ qui a chaque couple (état, action) associe une valeur d'espérance de récompense.

Il existe plusieurs algorithmes Actor-critic en RL, qui difèrent les uns des autres souvent par la fonction utilisée pour jouer le rôle de critic.

- ${Q(s,a)}$, qui va donner le Q Actor-critic
- ${G_t}$, qui est une somme discounted des récompenses futures à partir de ${t}$, et qui va donner REINFORCE
- ${A(s,a)}$ la fonction avantage, qui va donner Advantage actor-critic, que nous allons justement voir.

## A2C (Advantage Actor-Critic)

**A2C** (pour advantage actor critic) introduit une fonction avantage notée ${A(s,a)}$ qui est: _combien est-il avantageux, quand je suis dans l'état ${s}$, de choisir l'action ${a}$?_. 

On part en fait du principe que ${Q(s,a) = V(s) + A(s,a)}$. Puisque ${V(s)}$ est une estimation de la récompense moyenne à long terme à partir de ${s}$, y ajouter ${A(s,a)}$ (qui peut être une valeur négative) qui représente la différence par rapport à la récompense moyenne qu'a le fait de choisir l'action ${a}$ permet de retrouver ${Q(s,a)}$ la fonction qui a un couple ${(s,a)}$ associe la récompense moyenne à long terme.

On a donc aussi: ${A(s,a) = Q(s,a) - V(s)}$

L'avantage d'avoir une fonction advantage à apprendre plutôt qu'une fonction Q, est qu'elle permet de réduire la variance du modèle, en introduisant de plus petites valeurs.

In [37]:
import torch
import torch.nn as nn

class ActorCritic(nn.Module):
    def __init__(self, input_dim, num_actions, hidden_size):
        super(ActorCritic, self).__init__()

        self.num_actions = num_actions

        self.critic = nn.Sequential(nn.Linear(input_dim, hidden_size), nn.Tanh(),
            nn.Linear(hidden_size, 1))

        self.actor = nn.Sequential(nn.Linear(input_dim, hidden_size), nn.Tanh(), 
            nn.Linear(hidden_size, num_actions))

    def forward(self, observation):
        value = self.critic(observation)
        d = 1
        if len(observation.size()) == 1:
            d = 0
        policy_s = nn.functional.softmax(self.actor(observation), dim=d)

        return value, policy_s

## Appliquons cela sur un petit jeu simple, Mountain Car 

Doc et code source <a href="https://github.com/openai/gym/wiki/MountainCar-v0">ici</a>

Ce jeu est disponible dans l'environnement gym. Le principe est plutôt simple: faire grimper une colline à une voiture. La voiture n'est pas capable de monter en une seule fois: il faut avancer et reculer au fur et à mesure pour prendre de l'élan et réussir à monter. Le jeu se termine quand la voiture a atteint le drapeau, ou si la voiture n'a pas réussi après 200 itérations

Ce jeu est modélisé par des états codés comme un vecteur en 2 dimensions modélisant 2 mesures physiques: position de la voiture en x, et vitesse. Trois actions possible: bouger d'un cran vers la droite, bouger d'un cran vers la gauche, ne rien faire.

<img src="images/mountaincar.gif" width="500">

Configurons l'environnement Mountain Car-V0 avec gym:

In [38]:
import gym

env = gym.make('MountainCar-v0')
outdir = 'TP5/MountainCar-v0/A2C-agent-results'
envm = gym.wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
env.seed(0)
env.verbose = False

state = env.reset()
print("observation shape: ", state.shape)
print("action space: ", env.action_space.n)

observation shape:  (2,)
action space:  3


In [39]:
import numpy as np
import time

NUM_EPISODES = 1000
NUM_STEPS = 200 # 200 par défaut
GAMMA = 0.99

learning_rate = 0.001

hidden_layer_size = 64

input_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

actor_critic = ActorCritic(input_dim, action_dim, hidden_layer_size)
optimizer = torch.optim.Adam(actor_critic.parameters(), lr=learning_rate)

roi_arouf = []
nurmagomedov = []

## APPRENTISSAGE SUR DES SCENARIOS FINIS:
## JOUER UN SCENARIO COMPLET, PUIS APPRENDRE DESSUS

start = time.time()
# jouer un NUM_EPISODES "parties"
for episode in range(NUM_EPISODES):
    logprobs = []
    values = []
    rewards = []
    
    state = env.reset()
    # jouer au max NUM_STEPS "coups" pour cette partie
    # ... avec la politique actuelle bien sûr
    start_episode = time.time()
    for steps in range(NUM_STEPS):
        value, policy = actor_critic.forward(torch.from_numpy(state).float())
        value = value.detach().numpy()[0]
        policy_detach = policy.detach().numpy() 
        
        # on sample une action d'apres la distribution de la politique courante pour cet état
        chosen_action = np.random.choice(action_dim, p=policy_detach)
        # calculer la log-prob de la politique pour l'action choisie
        # pour pouvoir calculer le gradient par la suite
        log_prob = torch.log(policy.squeeze(0)[chosen_action])
        
        # effectuer l'action et récupérer le reward
        new_state, reward, done, _ = env.step(chosen_action)
        
        rewards.append(reward)
        values.append(value)
        logprobs.append(log_prob)
        state = new_state
        
        if done or steps == NUM_STEPS-1:
            # Si on est à la fin de l'épisode...
            Qval, _ = actor_critic.forward(torch.from_numpy(new_state).float())
            Qval = Qval.detach().numpy()[0]
            
            roi_arouf.append(np.sum(rewards))
            nurmagomedov.append(steps)

    # compute Q values
    Qvals = np.zeros_like(values)
    for t in reversed(range(len(rewards))):
        Qval = rewards[t] + GAMMA * Qval  #etape 2 update Qval 
        Qvals[t] = Qval
        
    values = torch.FloatTensor(values)
    Qvals = torch.FloatTensor(Qvals)
    logprobs = torch.stack(logprobs)
        
    advantage = Qvals - values #calcul de A (étape 3)
    
    ## Apprentissage
    
    # compute policy loss 
    actor_loss = (-logprobs * advantage).mean()
    # compute value loss (mse loss)
    critic_loss = advantage.pow(2).mean()
        
    ac_loss = actor_loss + critic_loss

    optimizer.zero_grad()
    ac_loss.backward()
    optimizer.step()
    
    if episode % 10 == 0:
        print("episode {}".format(episode)+" done in {0:.2f}s".format(time.time() - start_episode)+
             " reward: ", np.sum(rewards))
    
print("done in %f" % (time.time() - start))

episode 0 done in 0.23s reward:  -200.0
episode 10 done in 0.22s reward:  -200.0
episode 20 done in 0.25s reward:  -200.0
episode 30 done in 0.22s reward:  -200.0
episode 40 done in 0.23s reward:  -200.0


KeyboardInterrupt: 